In [1]:
pip install deepctr

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.2/141.2 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 55.4 MB/s eta 0:00:00
  Attempting uninstall: h5py
    Found existing installation: h5py 3.8.0
    Uninstalling h5py-3.8.0:
      Successfully uninstalled h5py-3.8.0


In [2]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from deepctr.models import xDeepFM
from deepctr.feature_column import  SparseFeat,get_feature_names


In [3]:
data=pd.read_csv("/content/criteo_sample.txt")
data

,label,I1,I2,I3,I4,I5,I6,I7,I8,I9,...,C17,C18,C19,C20,C21,C22,C23,C24,C25,C26
0,0,NaN,3,260.0,NaN,17668.0,NaN,NaN,33.0,NaN,...,e5ba7672,87c6f83c,NaN,NaN,0429f84b,NaN,3a171ecb,c0d61a5c,NaN,NaN
1,0,NaN,-1,19.0,35.0,30251.0,247.0,1.0,35.0,160.0,...,d4bb7bd8,6fc84bfb,NaN,NaN,5155d8a3,NaN,be7c41b4,ded4aac9,NaN,NaN
2,0,0.0,0,2.0,12.0,2013.0,164.0,6.0,35.0,523.0,...,e5ba7672,675c9258,NaN,NaN,2e01979f,NaN,bcdee96c,6d5d1302,NaN,NaN
3,0,NaN,13,1.0,4.0,16836.0,200.0,5.0,4.0,29.0,...,e5ba7672,52e44668,NaN,NaN,e587c466,NaN,32c7478e,3b183c5c,NaN,NaN
4,0,0.0,0,104.0,27.0,1990.0,142.0,4.0,32.0,37.0,...,e5ba7672,25c88e42,21ddcdc9,b1252a9d,0e8585d2,NaN,32c7478e,0d4a6d1a,001f3601,92c878de
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,0,NaN,0,113.0,3.0,3036.0,575.0,2.0,3.0,214.0,...,07c540c4,9880032b,21ddcdc9,5840adea,34cc61bb,c9d4222a,32c7478e,e5ed7da2,ea9a246c,984e0db0
196,1,0.0,1,1.0,1.0,1607.0,12.0,1.0,12.0,15.0,...,1e88c74f,3972b4ed,NaN,NaN,d1aa4512,NaN,32c7478e,9257f75f,NaN,NaN
197,1,1.0,0,6.0,3.0,0.0,0.0,19.0,3.0,3.0,...,3486227d,5aed7436,54591762,a458ea53,4a2c3526,NaN,32c7478e,1793a828,e8b83407,1a02cbe1
198,0,0.0,22,6.0,22.0,203.0,153.0,80.0,18.0,508.0,...,3486227d,13145934,55dd3565,5840adea,bf647035,NaN,32c7478e,1481ceb4,e8b83407,988b0775


In [4]:
sparse_features = ['C' + str(i) for i in range(1, 27)]
dense_features = ['I'+str(i) for i in range(1, 14)]

In [5]:
data[sparse_features] = data[sparse_features].fillna('-1', )
data[dense_features] = data[dense_features].fillna(0,)

In [6]:
target = ['label']

In [7]:
def encoding(data,feat,encoder):
    data[feat] = encoder.fit_transform(data[feat])

In [8]:
[encoding(data,feat,LabelEncoder()) for feat in sparse_features]

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [9]:
mms = MinMaxScaler(feature_range=(0,1))
data[dense_features] = mms.fit_transform(data[dense_features])

In [10]:
sparse_feature_columns = [SparseFeat(feat, vocabulary_size=data[feat].nunique(),embedding_dim=4) \
for i,feat in enumerate(sparse_features)]

In [11]:
from deepctr.feature_column import  DenseFeat

In [12]:
dense_feature_columns = [DenseFeat(feat, 1) for feat in dense_features]


In [13]:
dnn_feature_columns = sparse_feature_columns + dense_feature_columns

In [14]:
linear_feature_columns = sparse_feature_columns + dense_feature_columns

feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns)

In [15]:
train, test = train_test_split(data, test_size=0.2)

train_model_input = {name:train[name].values for name in feature_names}
test_model_input = {name:test[name].values for name in feature_names}

In [16]:
model = xDeepFM(linear_feature_columns, dnn_feature_columns, dnn_hidden_units=(256, 256),\
cin_layer_size=(128, 128), \
cin_split_half=True, cin_activation='relu'\
,l2_reg_linear=1e-05,\
l2_reg_embedding=1e-05, l2_reg_dnn=0, l2_reg_cin=0, \
seed=1024, dnn_dropout=0,dnn_activation='relu', \
dnn_use_bn=False, task='binary')

In [17]:
model.compile("adam", "binary_crossentropy",metrics=['binary_crossentropy'], )

In [18]:
history = model.fit(train_model_input, train[target].values,
                    batch_size=256, epochs=10, verbose=2, validation_split=0.2, )

Epoch 1/10
1/1 - 14s - loss: 0.7540 - binary_crossentropy: 0.7540 - val_loss: 0.6691 - val_binary_crossentropy: 0.6691
Epoch 2/10
1/1 - 0s - loss: 0.7151 - binary_crossentropy: 0.7150 - val_loss: 0.6515 - val_binary_crossentropy: 0.6514
Epoch 3/10
1/1 - 0s - loss: 0.6801 - binary_crossentropy: 0.6801 - val_loss: 0.6363 - val_binary_crossentropy: 0.6363
Epoch 4/10
1/1 - 0s - loss: 0.6481 - binary_crossentropy: 0.6480 - val_loss: 0.6238 - val_binary_crossentropy: 0.6238
Epoch 5/10
1/1 - 0s - loss: 0.6187 - binary_crossentropy: 0.6186 - val_loss: 0.6141 - val_binary_crossentropy: 0.6140
Epoch 6/10
1/1 - 0s - loss: 0.5916 - binary_crossentropy: 0.5916 - val_loss: 0.6070 - val_binary_crossentropy: 0.6070
Epoch 7/10
1/1 - 0s - loss: 0.5670 - binary_crossentropy: 0.5670 - val_loss: 0.6029 - val_binary_crossentropy: 0.6029
Epoch 8/10
1/1 - 0s - loss: 0.5449 - binary_crossentropy: 0.5448 - val_loss: 0.6019 - val_binary_crossentropy: 0.6019
Epoch 9/10
1/1 - 0s - loss: 0.5251 - binary_crossentrop

In [19]:
pred_ans_xdeep = model.predict(test_model_input, batch_size=256)


In [20]:
from sklearn.metrics import mean_squared_error

In [21]:
mean_squared_error(test["label"], pred_ans_xdeep)

0.20767929949302055

In [22]:
pred_ans_xdeep

array([[0.30340707],
       [0.28074753],
       [0.28099436],
       [0.29553822],
       [0.3028733 ],
       [0.26976767],
       [0.28950632],
       [0.31198928],
       [0.24994944],
       [0.30933407],
       [0.23119983],
       [0.29478168],
       [0.2919901 ],
       [0.21418683],
       [0.23255111],
       [0.314953  ],
       [0.30957195],
       [0.31355318],
       [0.302804  ],
       [0.3163681 ],
       [0.30711162],
       [0.29721227],
       [0.1866965 ],
       [0.07810273],
       [0.22673441],
       [0.29898402],
       [0.14504322],
       [0.23859832],
       [0.27369377],
       [0.23533563],
       [0.27831677],
       [0.21016298],
       [0.3253462 ],
       [0.27649847],
       [0.30232233],
       [0.2901496 ],
       [0.25320902],
       [0.328436  ],
       [0.2959744 ],
       [0.31697345]], dtype=float32)

In [23]:
from deepctr.models import DeepFM

In [24]:
model1 = DeepFM(linear_feature_columns, dnn_feature_columns, dnn_hidden_units=(256, 256),\
l2_reg_linear=1e-05,\
l2_reg_embedding=1e-05, l2_reg_dnn=0, \
seed=1024, dnn_dropout=0,dnn_activation='relu', \
dnn_use_bn=False, task='binary')

In [25]:
model1.compile("adam", "binary_crossentropy",metrics=['binary_crossentropy'], )

In [26]:
history = model1.fit(train_model_input, train[target].values,
                    batch_size=256, epochs=10, verbose=2, validation_split=0.2, )

Epoch 1/10
1/1 - 14s - loss: 0.6413 - binary_crossentropy: 0.6413 - val_loss: 0.6625 - val_binary_crossentropy: 0.6624
Epoch 2/10
1/1 - 0s - loss: 0.6201 - binary_crossentropy: 0.6200 - val_loss: 0.6544 - val_binary_crossentropy: 0.6543
Epoch 3/10
1/1 - 0s - loss: 0.6005 - binary_crossentropy: 0.6005 - val_loss: 0.6477 - val_binary_crossentropy: 0.6476
Epoch 4/10
1/1 - 0s - loss: 0.5825 - binary_crossentropy: 0.5825 - val_loss: 0.6420 - val_binary_crossentropy: 0.6419
Epoch 5/10
1/1 - 0s - loss: 0.5654 - binary_crossentropy: 0.5654 - val_loss: 0.6375 - val_binary_crossentropy: 0.6375
Epoch 6/10
1/1 - 0s - loss: 0.5491 - binary_crossentropy: 0.5491 - val_loss: 0.6342 - val_binary_crossentropy: 0.6342
Epoch 7/10
1/1 - 0s - loss: 0.5333 - binary_crossentropy: 0.5333 - val_loss: 0.6322 - val_binary_crossentropy: 0.6322
Epoch 8/10
1/1 - 0s - loss: 0.5181 - binary_crossentropy: 0.5181 - val_loss: 0.6317 - val_binary_crossentropy: 0.6316
Epoch 9/10
1/1 - 0s - loss: 0.5033 - binary_crossentrop

In [27]:
pred_ans_deep = model1.predict(test_model_input, batch_size=256)

In [28]:
mean_squared_error(test["label"], pred_ans_deep)

0.20296801524184924

In [29]:
pred_ans_deep

array([[0.35608587],
       [0.30707148],
       [0.1522691 ],
       [0.34537652],
       [0.2675847 ],
       [0.1609956 ],
       [0.3040123 ],
       [0.36655435],
       [0.20463629],
       [0.3720277 ],
       [0.19841757],
       [0.30275103],
       [0.25469127],
       [0.16085657],
       [0.13891542],
       [0.27056175],
       [0.34667084],
       [0.38472518],
       [0.3241339 ],
       [0.27326018],
       [0.3406915 ],
       [0.3742547 ],
       [0.13018523],
       [0.08815108],
       [0.22246644],
       [0.3032702 ],
       [0.17075075],
       [0.33900157],
       [0.3244718 ],
       [0.33276832],
       [0.30596182],
       [0.20188454],
       [0.39137614],
       [0.1844284 ],
       [0.26107743],
       [0.35396448],
       [0.3230076 ],
       [0.15291172],
       [0.32529867],
       [0.36473325]], dtype=float32)

In [30]:
import deepctr
print(deepctr.__version__)

0.9.3


In [31]:
from deepctr.models import AFM


In [32]:
dir(deepctr.models)

['AFM',
 'AutoInt',
 'BST',
 'CCPM',
 'DCN',
 'DCNMix',
 'DIEN',
 'DIFM',
 'DIN',
 'DSIN',
 'DeepFEFM',
 'DeepFM',
 'EDCN',
 'ESMM',
 'FGCNN',
 'FLEN',
 'FNN',
 'FiBiNET',
 'FwFM',
 'IFM',
 'MLR',
 'MMOE',
 'NFM',
 'ONN',
 'PLE',
 'PNN',
 'SharedBottom',
 'WDL',
 '__all__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 'afm',
 'autoint',
 'ccpm',
 'dcn',
 'dcnmix',
 'deepfefm',
 'deepfm',
 'difm',
 'edcn',
 'fgcnn',
 'fibinet',
 'flen',
 'fnn',
 'fwfm',
 'ifm',
 'mlr',
 'multitask',
 'nfm',
 'onn',
 'pnn',
 'sequence',
 'wdl',
 'xDeepFM',
 'xdeepfm']

In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error, roc_auc_score
from sklearn.model_selection import train_test_split
from deepctr.models import FM, DeepFM, NFFM, AFM
from deepctr.inputs import SparseFeat, DenseFeat, get_feature_names

# generate sample data
np.random.seed(0)
feature_dim = 5
sample_size = 10000
sparse_features = ['feat_' + str(i) for i in range(1, 4)]
dense_features = ['feat_' + str(i) for i in range(4, feature_dim + 1)]
data = {
    'feat_' + str(i): np.random.randint(0, 10, sample_size) for i in range(1, feature_dim + 1)
}
data['label'] = np.random.randint(0, 2, sample_size)

# create feature columns
feature_columns = [
    SparseFeat(feat, vocabulary_size=data[feat].max() + 1, embedding_dim=4)
    for feat in sparse_features
] + [
    DenseFeat(feat, 1)
    for feat in dense_features
]

# create train and test data
train_data, test_data = train_test_split(pd.DataFrame(data), test_size=0.2)

# create input tensors
train_sparse_features = [train_data[feat].values for feat in sparse_features]
train_dense_features = np.array(train_data[dense_features].values, dtype=np.float32)
train_labels = train_data['label'].values.astype(np.float32)

test_sparse_features = [test_data[feat].values for feat in sparse_features]
test_dense_features = np.array(test_data[dense_features].values, dtype=np.float32)
test_labels = test_data['label'].values.astype(np.float32)

# create model and train on train data
models = [
    FM(feature_columns),
    DeepFM(feature_columns),
    NFFM(feature_columns),
    AFM(feature_columns),
]
for model in models:
    model.compile(optimizer='adam', loss='mse', metrics=['AUC'])
    model.fit(
        x=[train_sparse_features, train_dense_features],
        y=train_labels,
        epochs=10,
        batch_size=32,
        validation_data=(
            [test_sparse_features, test_dense_features],
            test_labels,
        ),
        verbose=0,
    )

# evaluate models on test data
for i, model in enumerate(models):
    print('Model', i+1, ':')
    y_pred = model.predict([test_sparse_features, test_dense_features])
    mse = mean_squared_error(test_labels, y_pred)
    auc = roc_auc_score(test_labels, y_pred)
    print('  Mean Squared Error:', mse)
    print('  AUC:', auc)